# Treinamento de CNN do Zero - Parte 3: Avaliação e Comparação

Este notebook implementa a terceira parte do treinamento de uma Rede Neural Convolucional (CNN) do zero para classificar imagens nas mesmas categorias que usamos nos modelos YOLO.

Nesta terceira parte, vamos focar na avaliação do modelo treinado e na comparação com os modelos YOLO.

## 1. Configuração do Ambiente

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente.

In [ ]:
# Verificar se o ambiente já foi configurado
import os
import sys

# Se o ambiente ainda não foi configurado, execute o setup_env.sh
if not os.path.exists('../yolov5'):
    print("Configurando o ambiente com setup_env.sh...")
    !chmod +x ../setup_env.sh
    !../setup_env.sh
else:
    print("Ambiente já configurado.")

# Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import yaml
import time
import random
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

## 2. Carregamento dos Dados e Modelo Treinado

Vamos carregar os dados e o modelo treinado nas partes anteriores.

In [ ]:
# Verificar se os arquivos necessários existem
if not os.path.exists('../models/cnn/cnn_best.pt'):
    print("❌ Arquivo '../models/cnn/cnn_best.pt' não encontrado. Execute a Parte 2 primeiro.")
else:
    print("✅ Arquivo '../models/cnn/cnn_best.pt' encontrado.")

if not os.path.exists('../models/cnn/cnn_categories.txt'):
    print("❌ Arquivo '../models/cnn/cnn_categories.txt' não encontrado. Execute a Parte 1 primeiro.")
    # Usar categorias padrão
    categories = ['apple', 'banana']
else:
    print("✅ Arquivo '../models/cnn/cnn_categories.txt' encontrado.")
    # Carregar categorias
    with open('../models/cnn/cnn_categories.txt', 'r') as f:
        categories = [line.strip() for line in f.readlines()]
    print(f"Categorias: {categories}")

# Verificar se as métricas de treinamento foram salvas
if not os.path.exists('../models/cnn/cnn_training_metrics.npy'):
    print("❌ Arquivo '../models/cnn/cnn_training_metrics.npy' não encontrado. Execute a Parte 2 primeiro.")
    # Criar métricas vazias
    training_metrics = {
        'train_losses': [],
        'train_accs': [],
        'val_losses': [],
        'val_accs': [],
        'training_time': 0
    }
else:
    print("✅ Arquivo '../models/cnn/cnn_training_metrics.npy' encontrado.")
    # Carregar métricas
    training_metrics = np.load('../models/cnn/cnn_training_metrics.npy', allow_pickle=True).item()
    print(f"Tempo de treinamento: {training_metrics['training_time']:.2f} segundos")

# Definir diretório de teste
test_dir = '../dataset/test/images'

# Verificar se o diretório existe
if not os.path.exists(test_dir):
    print(f"❌ Diretório não encontrado: {test_dir}")
else:
    print(f"✅ Diretório encontrado: {test_dir}")
    print(f"   Número de imagens: {len([f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))])}")

### 2.1 Recriação do Dataset de Teste

Vamos recriar o dataset de teste para avaliação.

In [ ]:
# Definir transformações para as imagens
test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Classe personalizada para o dataset
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, categories, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.categories = categories
        
        # Listar todas as imagens
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))]
        
        # Determinar a classe de cada imagem com base no nome do arquivo
        self.labels = []
        for img_file in self.img_files:
            # Assumindo que o nome do arquivo começa com o nome da categoria
            # Por exemplo: categoria_a_001.jpg -> categoria_a
            for i, category in enumerate(categories):
                if category.lower() in img_file.lower():
                    self.labels.append(i)
                    break
            else:
                # Se não encontrar a categoria no nome do arquivo, usar a primeira categoria
                self.labels.append(0)
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Criar dataset de teste
test_dataset = CustomImageDataset(test_dir, categories, transform=test_transforms)

# Criar dataloader
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Verificar o dataset
print(f"Tamanho do dataset de teste: {len(test_dataset)}")

# Verificar a distribuição das classes
test_labels = test_dataset.labels
print("\nDistribuição das classes no conjunto de teste:")
for i, category in enumerate(categories):
    count = test_labels.count(i)
    print(f"  - {category}: {count} ({count/len(test_labels)*100:.1f}%)")

### 2.2 Recriação do Modelo

Vamos recriar o modelo CNN e carregar os pesos treinados.

In [ ]:
# Definir a arquitetura da CNN
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        
        # Camadas convolucionais
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Camadas de pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Camadas de batch normalization
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        
        # Camadas fully connected
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Bloco 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        
        # Bloco 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        
        # Bloco 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Bloco 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Definir o dispositivo (GPU se disponível, senão CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Criar o modelo
model = CustomCNN(num_classes=len(categories))

# Carregar os pesos treinados
if os.path.exists('../models/cnn/cnn_best.pt'):
    model.load_state_dict(torch.load('../models/cnn/cnn_best.pt'))
    print("Pesos treinados carregados com sucesso.")
else:
    print("Pesos treinados não encontrados. Execute a Parte 2 primeiro.")

# Mover o modelo para o dispositivo
model = model.to(device)
model.eval()

## 3. Avaliação do Modelo no Conjunto de Teste

Vamos avaliar o desempenho do modelo no conjunto de teste.

In [ ]:
# Função para validar o modelo
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Avaliando"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Estatísticas
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

# Definir função de perda
criterion = nn.CrossEntropyLoss()

# Avaliar no conjunto de teste
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f"Teste - Perda: {test_loss:.4f}, Acurácia: {test_acc:.4f}")

# Calcular métricas detalhadas
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.tight_layout()
plt.show()

## 4. Visualização de Predições

Vamos visualizar algumas predições do modelo no conjunto de teste.

In [ ]:
# Função para fazer predições em uma imagem
def predict_image(model, image_path, transform, device, categories):
    # Carregar a imagem
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Aplicar transformações
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Medir o tempo de inferência
    start_time = time.time()
    
    # Fazer a predição
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = F.softmax(outputs, dim=1)[0]
        _, predicted = torch.max(outputs, 1)
    
    # Calcular o tempo de inferência
    inference_time = time.time() - start_time
    
    # Obter a classe predita e a probabilidade
    predicted_class = categories[predicted.item()]
    probability = probabilities[predicted.item()].item()
    
    return image, predicted_class, probability, inference_time, probabilities.cpu().numpy()

# Obter imagens de teste
test_img_dir = '../dataset/test/images'
test_img_files = [os.path.join(test_img_dir, f) for f in os.listdir(test_img_dir) if f.endswith(('.jpg', '.jpeg', '.png', '.avif'))]

# Selecionar algumas imagens aleatórias
random.seed(42)  # Para reprodutibilidade
sample_imgs = random.sample(test_img_files, min(4, len(test_img_files)))

# Fazer predições e visualizar
plt.figure(figsize=(15, 12))
inference_times = []

for i, img_path in enumerate(sample_imgs):
    # Fazer a predição
    image, predicted_class, probability, inference_time, probabilities = predict_image(
        model, img_path, test_transforms, device, categories
    )
    inference_times.append(inference_time)
    
    # Mostrar a imagem e a predição
    plt.subplot(len(sample_imgs), 2, i*2+1)
    plt.imshow(image)
    plt.title(f"Imagem: {os.path.basename(img_path)}")
    plt.axis('off')
    
    # Mostrar as probabilidades
    plt.subplot(len(sample_imgs), 2, i*2+2)
    plt.barh(categories, probabilities)
    plt.title(f"Predição: {predicted_class} ({probability:.2f}) - {inference_time:.3f}s")
    plt.xlim(0, 1)
    plt.tight_layout()

plt.tight_layout()
plt.show()

# Calcular o tempo médio de inferência
avg_inference_time = np.mean(inference_times)
print(f"Tempo médio de inferência: {avg_inference_time:.4f} segundos")

## 5. Comparação com os Modelos YOLO

Vamos comparar o desempenho da CNN com os modelos YOLO (customizado e tradicional).

In [ ]:
# Criar um DataFrame para comparação
comparison_data = {
    'Modelo': ['CNN do Zero', 'YOLO Customizado (30 épocas)', 'YOLO Customizado (60 épocas)', 'YOLO Tradicional'],
    'Acurácia': [accuracy, 0.625, 0.625, 0.75],  # Valores obtidos dos resultados dos modelos
    'Precisão': [precision, 0.40501, 0.89753, 0.48],
    'Recall': [recall, 0.625, 0.625, 0.75],
    'F1-Score': [f1, 0.54109, 0.76108, 0.60],
    'Tempo de Inferência (s)': [avg_inference_time, 0.0030, 0.0008, 0.0736],
    'Tempo de Treinamento (s)': [training_metrics['training_time'], 3600, 7200, 0]  # Estimativa para os modelos YOLO
}

# Criar o DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Exibir o DataFrame
display(comparison_df)

## 6. Análise Comparativa

Vamos analisar e comparar os resultados dos diferentes modelos.

### Comparação de Desempenho

Com base nos resultados obtidos, podemos comparar o desempenho da CNN treinada do zero com os modelos YOLO:

1. **Precisão na Classificação vs. Detecção**:
   - A CNN é um modelo de classificação, enquanto o YOLO é um modelo de detecção de objetos.
   - A CNN classifica a imagem inteira, enquanto o YOLO detecta e classifica objetos específicos na imagem.
   - Para tarefas de classificação simples, a CNN pode ser mais eficiente, enquanto para detecção de objetos em cenas complexas, o YOLO é mais adequado.

2. **Tempo de Treinamento**:
   - A CNN geralmente requer menos tempo de treinamento do que o YOLO, pois tem menos parâmetros e uma arquitetura mais simples.
   - O YOLO customizado requer mais tempo de treinamento, especialmente com mais épocas.

3. **Tempo de Inferência**:
   - A CNN geralmente tem um tempo de inferência menor do que o YOLO, pois não precisa detectar objetos, apenas classificar a imagem inteira.
   - O YOLO tradicional pode ser mais rápido que o YOLO customizado devido a otimizações específicas.

4. **Facilidade de Uso**:
   - A CNN é mais fácil de implementar e treinar do que o YOLO, pois tem uma arquitetura mais simples.
   - O YOLO requer mais configuração e ajuste de hiperparâmetros.

5. **Aplicabilidade**:
   - A CNN é mais adequada para tarefas de classificação simples, onde a imagem contém apenas um objeto ou onde a classificação da imagem inteira é suficiente.
   - O YOLO é mais adequado para tarefas de detecção de objetos em cenas complexas, onde é necessário localizar e classificar múltiplos objetos.

### Conclusões

1. **Quando usar a CNN**:
   - Quando a tarefa é de classificação simples.
   - Quando o tempo de treinamento e inferência são críticos.
   - Quando os recursos computacionais são limitados.

2. **Quando usar o YOLO customizado**:
   - Quando a tarefa é de detecção de objetos específicos.
   - Quando é necessário localizar e classificar múltiplos objetos em uma imagem.
   - Quando a precisão na detecção é mais importante que o tempo de inferência.

3. **Quando usar o YOLO tradicional**:
   - Quando as categorias de interesse estão bem representadas no COCO.
   - Quando não há tempo ou recursos para treinar um modelo customizado.
   - Quando é necessário detectar uma variedade de objetos diferentes.

Em resumo, a escolha entre CNN e YOLO depende da natureza da tarefa, dos recursos disponíveis e dos requisitos de desempenho. Para tarefas de classificação simples, a CNN pode ser mais eficiente, enquanto para detecção de objetos em cenas complexas, o YOLO é mais adequado.